In [0]:
%pip install langchain accelerate transformers tensorflow  -q


Python interpreter will be restarted.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc3 3.11.5 requires numpy<1.22.2,>=1.15.0, but you have numpy 1.26.4 which is incompatible.
scipy 1.7.3 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.26.4 which is incompatible.
Python interpreter will be restarted.


In [0]:
# IMPORTS
import os
from langchain import PromptTemplate, LLMChain
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, AutoModelForQuestionAnswering
import torch
import boto3
from io import BytesIO
from langchain import HuggingFacePipeline

/databricks/python/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [0]:
def list_folders_in_folder(bucket_name, folder_path):
    """
    List the folders directly under a particular folder (prefix) in an S3 bucket.
    
    Args:
        bucket_name (str): Name of the S3 bucket.
        folder_path (str): Path to the folder (prefix) in the bucket.
        
    Returns:
        list: List of folder names in the specified folder.
    """
    # Create an S3 client
    s3 = boto3.client('s3')
    
    # List objects in the folder (prefix) with delimiter set to '/'
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_path, Delimiter='/')
    
    # Extract folder names from the response
    folders_path = [prefix['Prefix'] for prefix in response.get('CommonPrefixes', [])]
    folders_in_folder = [prefix['Prefix'].split('/')[-2] for prefix in response.get('CommonPrefixes', [])]
    
    return folders_path, folders_in_folder

bucket_name = 'analyst-adhoc'
folder_path = 'Campaign_Snapshot_Canada/Test_Canada_Campaign_Snapshot_MiQ_LLM/'

folders_path, folders_in_folder = list_folders_in_folder(bucket_name, folder_path)

print(folders_path)
print(folders_in_folder)

['Campaign_Snapshot_Canada/Test_Canada_Campaign_Snapshot_MiQ_LLM/City/', 'Campaign_Snapshot_Canada/Test_Canada_Campaign_Snapshot_MiQ_LLM/Creative/', 'Campaign_Snapshot_Canada/Test_Canada_Campaign_Snapshot_MiQ_LLM/Demostats/', 'Campaign_Snapshot_Canada/Test_Canada_Campaign_Snapshot_MiQ_LLM/Eshopper/', 'Campaign_Snapshot_Canada/Test_Canada_Campaign_Snapshot_MiQ_LLM/Impressions_count/', 'Campaign_Snapshot_Canada/Test_Canada_Campaign_Snapshot_MiQ_LLM/Numeris/', 'Campaign_Snapshot_Canada/Test_Canada_Campaign_Snapshot_MiQ_LLM/Prizm/', 'Campaign_Snapshot_Canada/Test_Canada_Campaign_Snapshot_MiQ_LLM/Region/', 'Campaign_Snapshot_Canada/Test_Canada_Campaign_Snapshot_MiQ_LLM/Strategy/', 'Campaign_Snapshot_Canada/Test_Canada_Campaign_Snapshot_MiQ_LLM/Tactic/', 'Campaign_Snapshot_Canada/Test_Canada_Campaign_Snapshot_MiQ_LLM/Temporal/']
['City', 'Creative', 'Demostats', 'Eshopper', 'Impressions_count', 'Numeris', 'Prizm', 'Region', 'Strategy', 'Tactic', 'Temporal']


In [0]:
MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.2"
CACHE_DIR = "dbfs:/tmp/hf_mistral_cache"
ADVERTISER_FLAG = "Advertiser_flag=dzx5gjn-72lfq2o;jyptqpl;byl8rou;6ndxcjp-2023-05-01-2023-05-30/"

In [0]:
def list_parquet_files_in_folder(bucket_name, folder_path):
    """
    List the Parquet files in a folder (prefix) in an S3 bucket.
    
    Args:
        bucket_name (str): Name of the S3 bucket.
        folder_path (str): Path to the folder (prefix) in the bucket.
        
    Returns:
        list: List of Parquet file keys (paths) in the specified folder.
    """
    # Create an S3 client
    s3 = boto3.client('s3')
    
    # List objects in the folder (prefix)
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_path)
    
    # Filter Parquet files from the response
    parquet_files = [obj['Key'] for obj in response.get('Contents', []) if obj['Key'].endswith('.parquet')]
    
    return parquet_files


In [0]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(MODEL_ID,device_map="auto",cache_dir=CACHE_DIR)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [0]:
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer,
    max_new_tokens=512
)

# Create an instance of the HuggingFacePipeline, which wraps the question-answering pipeline
# with additional model-specific arguments (temperature and max_length)
llm = HuggingFacePipeline(
    pipeline=pipe,
    model_kwargs={"temperature": 0.2, "max_length": 2048},
)

In [0]:
template = """
You are an analyst reviewing data insights in the ad-tech domain. You will be provided with data columns and their corresponding values, representing various metrics and attributes from an advertising campaign. Your task is to narrate the insights derived from this data. You will be provided one row of the dataframe.

Row : {row}

Please narrate the insights for each relevant data point, providing analysis and interpretations based on the provided values.
Provide a short summary of the insights generated at the end.
"""

In [0]:
prompt = PromptTemplate(template=template, input_variables=['row'])
chain = LLMChain(prompt=prompt, llm=llm)

In [0]:
import pandas as pd
log = pd.DataFrame(columns=['Advertisement_Flag', 'Folder_name', 'URI', 'Parquet_contents', 'LLM_Response'])

In [0]:
import pyspark.sql.functions as F

for i in range(len(folders_in_folder)):
  PATH = f"{folders_path[i]}{ADVERTISER_FLAG}"
  parquet_files_in_folder = list_parquet_files_in_folder(bucket_name, PATH)
  if len(parquet_files_in_folder) != 0:
    p = parquet_files_in_folder[0]
    s3uri = f"s3://{bucket_name}/{p}"
    df = spark.read.parquet(s3uri)
    row = df.first()
    response = chain.invoke({'row': row})
    res = response['text']
    new_log = {
            'Advertisement_Flag': f'{ADVERTISER_FLAG}',
            'Folder_name': f'{folders_in_folder[i]}',
            'URI': f'{s3uri}',
            'Parquet_contents': f'{row}',
            'LLM_Response': f'{res}'
        }
  else:
    new_log = {
            'Advertisement_Flag': f'{ADVERTISER_FLAG}',
            'Folder_name': f'{folders_in_folder[i]}',
            'URI': f'{PATH}',
            'Parquet_contents': 'NULL',
            'LLM_Response': 'parquet file not found!'
        }
  log.loc[len(log)] = new_log

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [0]:
log.head()

,Advertisement_Flag,Folder_name,URI,Parquet_contents,LLM_Response
0,Advertiser_flag=dzx5gjn-72lfq2o;jyptqpl;byl8ro...,City,Campaign_Snapshot_Canada/Test_Canada_Campaign_...,NULL,parquet file not found!
1,Advertiser_flag=dzx5gjn-72lfq2o;jyptqpl;byl8ro...,Creative,s3://analyst-adhoc/Campaign_Snapshot_Canada/Te...,Row(creative_name='CA_TENA_DISPLAY_INCO_CA_APE...,\nInsights:\n1. The creative name 'CA_TENA_DIS...
2,Advertiser_flag=dzx5gjn-72lfq2o;jyptqpl;byl8ro...,Demostats,s3://analyst-adhoc/Campaign_Snapshot_Canada/Te...,"Row(catdesc='Households by Maintainer Age', de...",\nInsights:\n1. The given data represents the ...
3,Advertiser_flag=dzx5gjn-72lfq2o;jyptqpl;byl8ro...,Eshopper,s3://analyst-adhoc/Campaign_Snapshot_Canada/Te...,"Row(Category='Online Product Purchase', Descri...",\nInsights:\n1. The given row represents an ad...
4,Advertiser_flag=dzx5gjn-72lfq2o;jyptqpl;byl8ro...,Impressions_count,s3://analyst-adhoc/Campaign_Snapshot_Canada/Te...,"Row(advertiser='CA - Tena c.o Starcom Canada',...",\nInsights:\n1. Impression count display: The ...


In [0]:

log.to_csv("/dbfs/tmp/s3_insights_v2.csv")

In [0]:
print(log['LLM_Response'][1])


Insights:
1. The creative name 'CA_TENA_DISPLAY_INCO_CA_APEX Programmatic Display Video-CA-TENA-DD-INCO-Men-M45+-PRO-PROOX-PD-DCPM-TENA-Men-Acceptors-StandardBanner-Prop-EN-160x600-New-GLD000KJGC_230228_DCM_360691291' suggests that this is a programmatic display video ad campaign for TENA targeting men aged 45 and above, with a creative size of 160x600 pixels. The creative name also indicates that this is a new campaign with the ID '360691291'.

2. The 'impressions_display' metric indicates that this creative has been displayed 7 times. This means that the ad has been shown to potential viewers 7 times, but it does not necessarily mean that 7 unique individuals have seen the ad.

3. The 'clicks' metric is 0, indicating that no clicks have been recorded for this creative. This could mean that the ad is not resonating with the target audience or that it is not being shown to the right audience.

4. The 'total_conversions' metric is also 0, indicating that no conversions have been record

In [0]:
print(log['LLM_Response'][2])


Insights:
1. The given data represents the demographic segment 'Households by Maintainer Age' with a specific age range 'Maintainers 25 To 34'. This indicates that the campaign is targeting households where the primary decision-maker or maintainer is between the ages of 25 and 34.
2. The 'impressions_display' metric shows that this demographic segment was displayed 9 times in the campaign. This suggests that the campaign reached out to 9 households within this age group during the campaign period.
3. The 'clicks' metric is zero, indicating that no clicks were recorded for this demographic segment during the campaign. This could mean that the campaign did not resonate with this age group or that the ad creative did not effectively engage them.
4. The 'total_conversions' metric is also zero, suggesting that no conversions were made by this demographic segment during the campaign. This could be due to a lack of interest, ineffective targeting, or poor ad creative.

Summary:
The data sugg

In [0]:
print(log['LLM_Response'][3])


Insights:
1. The given row represents an advertising campaign category as 'Online Product Purchase' with a description 'Expected online spend [Nxt 12 Mths] - Government services - Less'. This indicates that the campaign is focused on reducing the expected online spend for government services in the next 12 months.
2. The 'impressions_display' value is 72. This metric represents the number of times the ad was displayed to users. A low number of impressions could mean that the ad is not reaching a large audience or that the targeting is not effective. However, in this case, it's not possible to make definitive conclusions based on this single data point.
3. The 'clicks' value is 0. This metric represents the number of times users have clicked on the ad. A low number of clicks could indicate that the ad is not resonating with the target audience or that the call-to-action is not clear. In this case, it's also not possible to make definitive conclusions based on this single data point.
4.

In [0]:
print(log['LLM_Response'][4])


Insights:
1. Impression count display: The advertiser 'CA - Tena c.o Starcom Canada' had a total of 2,789,888 impressions during the campaign period from May 1, 2023, to May 31, 2023. This indicates a significant reach for their advertising message, potentially leading to brand awareness and recall.

2. Clicks count: With a total of 2,450 clicks recorded during the campaign, it suggests that a considerable number of users were engaged enough by the ad to click on it. This engagement could translate into potential leads or sales, depending on the nature of the advertiser's business.

3. Click-through rate (CTR): The CTR of 0.0878% is a crucial metric to evaluate the effectiveness of the ad. A higher CTR indicates that a larger percentage of users who saw the ad clicked on it, suggesting that the ad was more appealing and relevant to the target audience. In this case, the CTR is relatively low, which could indicate room for improvement in the ad's targeting, messaging, or design.

Summa

In [0]:
print(log['LLM_Response'][5])


Insights:
1. The given row represents data for an advertising campaign in the 'Personal Care - Cosmetics' category, specifically for 'Frequency of Using' non-disposable shavers.
2. The 'impressions_display' metric indicates that the ad was displayed 60 times during the past week. This suggests a moderate level of engagement with the ad, as the number of impressions is not extremely high but still significant.
3. The 'clicks' metric is zero, indicating that no clicks were recorded for this ad during the past week. This could be due to various reasons such as low relevance, poor ad placement, or lack of interest from the target audience.
4. The 'total_conversions' metric is also zero, suggesting that no sales or conversions were generated from this ad during the past week. This could be due to the lack of clicks or the fact that the target audience was not in the purchasing mindset during the campaign period.
5. The 'population1_display' metric represents the estimated population that w